# Pairwise cosine similarities between embeddings throughout training

In [3]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
import matplotlib.pyplot as plt
import numpy as np

cuda


In [ ]:
# download embeds
from huggingface_hub import snapshot_download
snapshot_download(repo_id="amphora/pythia-12b-weights", 
                  local_dir='pythia-12b-weights',
                  local_dir_use_symlinks=False,
                  allow_patterns="*.pkl")

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into './embeds'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (10/10), 3.26 KiB | 222.00 KiB/s, done.


In [1]:

embedding_filename = "../embeds/pythia-12b.pth"
embedding_layer = torch.load(embedding_filename, map_location=torch.device(device))
embedding_weights = embedding_layer.weight.data
print(embedding_weights)

cuda
tensor([[ 0.0023, -0.0030,  0.0049,  ..., -0.0018,  0.0002, -0.0021],
        [ 0.0140, -0.0061,  0.0213,  ..., -0.0045, -0.0138, -0.0004],
        [ 0.0062, -0.0107,  0.0072,  ...,  0.0063,  0.0007, -0.0219],
        ...,
        [-0.0126, -0.0018, -0.0049,  ..., -0.0056,  0.0099,  0.0101],
        [ 0.0040,  0.0063, -0.0115,  ..., -0.0013, -0.0122,  0.0051],
        [ 0.0054, -0.0056, -0.0129,  ...,  0.0047, -0.0046, -0.0066]],
       device='cuda:0')


In [2]:
def cos_sim(embed_matrix):
    """
    Takes in embedding matrix at any step. ensure it is on cuda for MAXIMUM EFFICIENCY.
    Expected that the output cosine similarity matrix is of torch.Size([50688, 50688])
    """
    weights = embed_matrix.weight.data
    embedding_matrix_norm = weights / weights.norm(dim=1)[:, None]
    cosine_similarity = torch.mm(embedding_matrix_norm, embedding_matrix_norm.transpose(0,1))
    return cosine_similarity

In [5]:
import pickle

files = [
 './pythia-12b-weights/embed_only_0-29000.pkl',
 './pythia-12b-weights/embed_only_30000-69000.pkl',
 './pythia-12b-weights/embed_only_70000-109000.pkl',
 './pythia-12b-weights/embed_only_110000-143000.pkl'
]

output = {}
for filename in files:
    with open(filename, 'rb') as in_file:
        data = pickle.load(in_file)
        for step_idx, embeddings in data.items():
            sim_matrix = cos_sim(embeddings.to(device)).cpu()

torch.Size([50688, 50688])